In [1]:
import numpy as np
import pandas as pd

# Task 0
Read the dataset from csv file & perform data cleaning - remove all rows, which contains `?` in some columns.
Also check for data correctness (salary & salary $K).

In [2]:
df = pd.read_csv("data.adult.csv")

,Unnamed: 0,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary,salary K$
0,0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,40,United-States,<=50K,39
1,1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,United-States,<=50K,35
2,2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,<=50K,27
3,3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,<=50K,43
4,4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,<=50K,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,32556,27,Private,Assoc-acdm,Married-civ-spouse,Tech-support,Wife,White,Female,38,United-States,<=50K,36
32557,32557,40,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,40,United-States,>50K,173
32558,32558,58,Private,HS-grad,Widowed,Adm-clerical,Unmarried,White,Female,40,United-States,<=50K,40
32559,32559,22,Private,HS-grad,Never-married,Adm-clerical,Own-child,White,Male,20,United-States,<=50K,38


# Task 1
Print the count of men and women in the dataset.

In [16]:
# write your code for task 1 here

39.43354749885268


# Task 2
Find the average age of men in dataset

In [17]:
# write your code for task 2 here

39.43354749885268


# Task 3
Get the percentage of people from Poland (native-country)

In [22]:
# write your code for task 3 here

32561
0.18426952489174164


# Task 4
Get the mean and standard deviation of the age for people who earn > 50K per year. After this, get it for those who earn <= 50K.

In [30]:
# write your code for task 4 here

# Task 5
Check, if there are some people without higher education (education: Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters, Doctorate), but with > 50K salary

In [31]:
# write your code for task 5 here

3306

# Task 6
Get the statistics of age for each type of education. Use `groupby` and `describe` for this.

In [33]:
# write your code for task 6 here

,count,mean,std,min,25%,50%,75%,max
education,,,,,,,,
10th,933.0,37.429796,16.720713,17.0,22.00,34.0,52.0,90.0
11th,1175.0,32.355745,15.545485,17.0,18.00,28.0,43.0,90.0
12th,433.0,32.000000,14.334625,17.0,19.00,28.0,41.0,79.0
1st-4th,168.0,46.142857,15.615625,19.0,33.00,46.0,57.0,90.0
5th-6th,333.0,42.885886,15.557285,17.0,29.00,42.0,54.0,84.0
7th-8th,646.0,48.445820,16.092350,17.0,34.25,50.0,61.0,90.0
9th,514.0,41.060311,15.946862,17.0,28.00,39.0,54.0,90.0
Assoc-acdm,1067.0,37.381443,11.095177,19.0,29.00,36.0,44.0,90.0
Assoc-voc,1382.0,38.553546,11.631300,19.0,30.00,37.0,46.0,84.0


# Task 7
Compare the married and non-married men salaries. Who earns more? (>50K or <=50K)
Married men are those, whom `marital-status` starts with "Married". Others are not.

In [51]:
# write your code for task 7 here

Married men earn more than 50K: False
Not married men earn more than 50K: False


/var/folders/cv/jmpmmt854gq0ggsxxmc0g_tw0000gn/T/ipykernel_8002/4055146472.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  married_men_salary_earned_gt_50k_counts = married_men[df["salary"] == ">50K"].shape[0]
/var/folders/cv/jmpmmt854gq0ggsxxmc0g_tw0000gn/T/ipykernel_8002/4055146472.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  married_men_salary_earned_lte_50k_counts = married_men[df["salary"] == "<=50K"].shape[0]
/var/folders/cv/jmpmmt854gq0ggsxxmc0g_tw0000gn/T/ipykernel_8002/4055146472.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  non_married_men_salary_earned_gt_50k_counts = non_married_men[df["salary"] == ">50K"].shape[0]
/var/folders/cv/jmpmmt854gq0ggsxxmc0g_tw0000gn/T/ipykernel_8002/4055146472.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  non_married_men_salary_earned_lte_50k_counts = non_married_men[df["salary"] == "<=50K"].shape

# Task 8
Get the max hours per week some person works. How many people works the same amount of hours per week?

In [54]:
# write your code for task 8 here

85


# Task 9
Analyze the correlation between data in dataset. Understand connected fields in it and print highlight thier connection.

In [66]:
# write your code for task 9 here

                     age  hours-per-week  salary K$
age             1.000000        0.068756   0.201774
hours-per-week  0.068756        1.000000   0.196916
salary K$       0.201774        0.196916   1.000000
Age and Salary: There is a positive correlation between age and salary (0.201774).
This suggests that, on average, as a person's age increases, their salary tends to increase as well.
This connection could be related to career progression and experience.
--------------------------------------------------
Hours per Week and Salary:
There is a positive correlation between hours per week worked and salary (0.196916).
This indicates that individuals who work more hours per week tend to have higher salaries on average.
This connection might reflect higher earnings for individuals with more working hours.
--------------------------------------------------
Age and Hours per Week:
There is a relatively weak positive correlation between age and hours per week worked (0.068756).
This connect